In [ ]:
# | default_exp _index_file_generator

In [ ]:
# | export

from typing import *

import typer
from llama_index.readers.schema.base import Document
from llama_index import download_loader, GPTSimpleVectorIndex

from fastkafkachat._helper import get_all_links_from_website, get_service_context, write_compressed_json

In [ ]:
import time
from tempfile import TemporaryDirectory
from pathlib import Path

from typer.testing import CliRunner

from fastkafkachat._helper import load_compressed_json

In [ ]:
runner = CliRunner()

In [ ]:
# | export


def _index_website_data(
    start_url: str = "https://fastkafka.airt.ai",
    data_dir: str = "./data",
) -> None:
    """Extract and index website data from the given start URL.

    Args:
        start_url: The starting URL of the website. Defaults to "https://fastkafka.airt.ai".
        data_dir: The data directory path to save the index file generated by parsing all the website links.
    """
    urls = list(get_all_links_from_website(start_url))
    
    BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")
    loader = BeautifulSoupWebReader()
    documents = loader.load_data(urls=urls)

    service_context = get_service_context()
    index = GPTSimpleVectorIndex.from_documents(
        documents, service_context=service_context
    )
    write_compressed_json(index.save_to_string(), f"{data_dir}/website_index.json")

In [ ]:
with TemporaryDirectory() as d:
    data_path = Path(d) / "data"
    data_path.mkdir(parents=True)
    
    start_time = time.time()
    data_dir=f"{d}/data/" #website_index.json.gz
    documents = _index_website_data(data_dir=data_dir)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    
    index_json = load_compressed_json(f"{d}/data/website_index.json.gz")
    index = GPTSimpleVectorIndex.load_from_string(index_json)
    print(index)
    assert type(index) == GPTSimpleVectorIndex

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 68644 tokens


Elapsed time: 39.11979103088379 seconds
<llama_index.indices.vector_store.vector_indices.GPTSimpleVectorIndex object>


In [ ]:
# | export
_app = typer.Typer()


@_app.command(
    help="Recursively parses all anchor links found on the website and index website data from the given start URL",
)
def index_website_data(
    start_url: str = typer.Option(
        "https://fastkafka.airt.ai",
        help="The starting URL of the website",
    ),
    data_dir: str = typer.Option(
        "./data/",
        help="The data directory path to save the index file generated by parsing all the website links.",
    ),
) -> None:
    try:
        _index_website_data(
            start_url=start_url,
            data_dir=data_dir
        )
    except Exception as e:
        typer.secho(e, err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
result = runner.invoke(_app, ["--help"])

                                                                                                                   
 Usage: index-website-data [OPTIONS]                                                                               
                                                                                                                   

Recursively parses all anchor links found on the website and index website data from the given start URL

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --start-url                 TEXT  The starting URL of the website [default: https://fastkafka.airt.ai]          │
│ --data-dir                  TEXT  The data directory path to save the index file generated by parsing all the   │
│                                   website links.                                                                │
│                                   [default: ./data/]                                                            │
│ --install-completion              Install completion for the current shell.                                     │
│ --show-completion                 Show completion for the current shell, to copy it or customize the            │
│                                   installation.                                                                 │
│ --help                            Show this message and exit.                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯